#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CLIENT CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter CM360 To DV360 Bulk Creator Parameters
Allows bulk creating DV360 Insertion Orders and Line Items from CM360.
 1. Select <b>Load Partners</b>, then click <b>Save + Run</b>, then a sheet called DV Editor  will be created.
 1. In the <b>Partners</b> sheet tab, fill in <i>Filter</i> column then select <b>Load Advertisers</b>, click <b>Save + Run</b>.
 1. In the <b>Advertisers</b> sheet tab, fill in <i>Filter</i> column then select <b>Load Campaigns</b>, click <b>Save + Run</b>.
 1. In the <b>Campaigns</b> sheet tab, fill in <i>Filter</i> column, optional.
 1. Then select <b>Load Insertion Orders And Line Items</b>, click <b>Save + Run</b>.
 1. To update values, make changes on all <i>Edit</i> columns.
 1. Select <i>Preview</i>, then <b>Save + Run</b>.
 1. Check the <b>Audit</b> and <b>Preview</b> tabs to verify commit.
 1. To commit changes, select <i>Update</i>, then <b>Save + Run</b>.
 1. Check the <b>Success</b> and <b>Error</b> tabs.
 1. Update can be run multiple times.
 1. Update ONLY changes fields that do not match their original value.
 1. Insert operates only on Edit columns, ignores orignal value columns.
 1. Carefull when using drag to copy rows, values are incremented automatically.
 1. Modify audit logic by visting BigQuery and changing the views.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_dv': 'user',  # Credentials used for dv.
  'auth_cm': 'user',  # Credentials used for dv.
  'auth_sheet': 'user',  # Credentials used for sheet.
  'auth_bigquery': 'service',  # Credentials used for bigquery.
  'recipe_name': '',  # Name of Google Sheet to create.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'command': 'Load',  # Action to take.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute CM360 To DV360 Bulk Creator
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import Configuration
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      '__comment__': 'Ensure dataset exists.',
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','prefix': 'CM_To_DV_','kind': 'string','order': 2,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'drive': {
      '__comment__': 'Copy the default template to sheet with the recipe name',
      'auth': 'user',
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/1XjEHq-nEFMW8RVmCNJ-TVGvVcVBEADzjbvhmAvF04iQ/edit#gid=594912061',
        'destination': {'field': {'name': 'recipe_name','prefix': 'CM To DV ','kind': 'string','order': 3,'default': '','description': 'Name of Google Sheet to create.'}}
      }
    }
  },
  {
    'cm_to_dv': {
      '__comment': 'Depending on users choice, execute a different part of the solution.',
      'auth_dv': {'field': {'name': 'auth_dv','kind': 'authentication','order': 1,'default': 'user','description': 'Credentials used for dv.'}},
      'auth_cm': {'field': {'name': 'auth_cm','kind': 'authentication','order': 2,'default': 'user','description': 'Credentials used for dv.'}},
      'auth_sheets': {'field': {'name': 'auth_sheet','kind': 'authentication','order': 3,'default': 'user','description': 'Credentials used for sheet.'}},
      'auth_bigquery': {'field': {'name': 'auth_bigquery','kind': 'authentication','order': 4,'default': 'service','description': 'Credentials used for bigquery.'}},
      'sheet': {'field': {'name': 'recipe_name','prefix': 'CM To DV ','kind': 'string','order': 5,'default': '','description': 'Name of Google Sheet to create.'}},
      'dataset': {'field': {'name': 'recipe_slug','prefix': 'CM_To_DV_','kind': 'string','order': 6,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
      'command': {'field': {'name': 'command','kind': 'choice','choices': ['Clear','Load','Preview','Insert'],'order': 6,'default': 'Load','description': 'Action to take.'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(Configuration(project=CLOUD_PROJECT, client=CLIENT_CREDENTIALS, user=USER_CREDENTIALS, verbose=True), TASKS, force=True)
